# Solucion de Viga por el Metodo de la Flexibilidad
1. Definimos las cargas en la estructura isostatica equivalente:
Sistema Original = Sistema Empotrado + Sistema Equivalente
2. Definimos la matriz de flexibilidad no ensamblada
3. Definimos la matriz de transformacion y calculamos la matriz de flexibilidad ensamblada
4. Particionamos la matriz de flexibilidad y resolvemos el sistema de ecuaciones
5. Combinamos el sistema real y el equivalente para obtener la solucion del sistema original

In [42]:
# Importamos dependencias
import numpy as np
import math
import sympy as sp
from IPython.display import display, Latex

In [43]:
# Definimos funciones de apoyo
def matrizFlexibilidad(E,I,L):
    fm=sp.Matrix([[L**3/(3*E*I),L**2/(2*E*I)],
                 [L**2/(2*E*I),L/(E*I)]])
    return fm


## Matriz de Flexibilidad No Ensamblada

In [44]:
# Definimos variables simbolicas
E,I,L,P=sp.symbols('E I L P')
fm1=matrizFlexibilidad(E,I,L)
fm2=matrizFlexibilidad(E,I,L)

fm=sp.diag(fm1,fm2)

print('La matriz fm1 es:')
display(fm1)
print('La matriz fm2 es:')
display(fm2)
print('La matriz no ensamblada es:')
display(fm)


La matriz fm1 es:


Matrix([
[L**3/(3*E*I), L**2/(2*E*I)],
[L**2/(2*E*I),      L/(E*I)]])

La matriz fm2 es:


Matrix([
[L**3/(3*E*I), L**2/(2*E*I)],
[L**2/(2*E*I),      L/(E*I)]])

La matriz no ensamblada es:


Matrix([
[L**3/(3*E*I), L**2/(2*E*I),            0,            0],
[L**2/(2*E*I),      L/(E*I),            0,            0],
[           0,            0, L**3/(3*E*I), L**2/(2*E*I)],
[           0,            0, L**2/(2*E*I),      L/(E*I)]])

## Matriz de Transformacion y Matriz de Flexibilidad Ensamblada

In [45]:
BMS=sp.zeros(4,4)
BMS[:,0]=sp.Matrix([[-1/L],
                    [1],
                    [0],
                    [0]])
BMS[:,1]=sp.Matrix([[0],
                    [0],
                    [-1/L],
                    [1]])
BMS[:,2]=sp.Matrix([[-1/L],
                    [0],
                    [0],
                    [0]])
BMS[:,3]=sp.Matrix([[-1/L],
                    [1],
                    [1/L],
                    [0]])

print('La matriz de transformacion es:')
display(BMS)

FM=BMS.transpose()*fm*BMS

print('La matriz ensamblada es:')
display(FM)

La matriz de transformacion es:


Matrix([
[-1/L,    0, -1/L, -1/L],
[   1,    0,    0,    1],
[   0, -1/L,    0,  1/L],
[   0,    1,    0,    0]])

La matriz ensamblada es:


Matrix([
[ L/(3*E*I),         0, -L/(6*E*I),   L/(3*E*I)],
[         0, L/(3*E*I),          0,   L/(6*E*I)],
[-L/(6*E*I),         0,  L/(3*E*I),  -L/(6*E*I)],
[ L/(3*E*I), L/(6*E*I), -L/(6*E*I), 2*L/(3*E*I)]])

## Separamos las Matrices y Resolvemos el sistema de Ecuaciones
$[D_S]=[F_M][A_S]$

$
\begin{bmatrix}
D_J \\
D_Q
\end{bmatrix} =
\begin{bmatrix}
F_{JJ} & F_{JQ} \\
F_{QJ} & F_{QQ}
\end{bmatrix}
\begin{bmatrix}
A_J \\
A_Q
\end{bmatrix}
$

In [46]:
FJJ=FM[0:2,0:2]
FJQ=FM[0:2,2:4]
FQJ=FM[2:4,0:2]
FQQ=FM[2:4,2:4]

display(FJJ)
display(FJQ)
display(FQJ)
display(FQQ)

Matrix([
[L/(3*E*I),         0],
[        0, L/(3*E*I)]])

Matrix([
[-L/(6*E*I), L/(3*E*I)],
[         0, L/(6*E*I)]])

Matrix([
[-L/(6*E*I),         0],
[ L/(3*E*I), L/(6*E*I)]])

Matrix([
[ L/(3*E*I),  -L/(6*E*I)],
[-L/(6*E*I), 2*L/(3*E*I)]])

In [65]:
# Se resuelve el sistema de ecuaciones

DQ=sp.zeros(2,1)
print('El vector DQ es:')
display(DQ)

AS_load_node=sp.Matrix([[P*L],
                        [0],
                        [0],
                        []])
AS_load_FE=sp.Matrix([[],
                      [],
                      [],
                      []])
AS_load_EQ=-AS_load_FE

AJ_loadNudo=sp.Matrix([[P*L],
                       [0]])
AJ_loadEQ=sp.Matrix([[P*L/4-P*L/8],
                     [P*L/8]])
AJ_loadFE=-AJ_loadEQ


print('Los vectores de carga son:')
display(AJ_loadNudo)
display(AJ_loadEQ)
display(AJ_loadFE)

# Resolvemos para AQ
AQ=FQQ.inv()*(DQ-FQJ*(AJ_loadNudo+AJ_loadEQ))
AQ=AQ

print('El valor de las redundantes en el sistema equivalente es:')
display(AQ)



El vector DQ es:


Matrix([
[0],
[0]])

Los vectores de carga son:


Matrix([
[L*P],
[  0]])

Matrix([
[L*P/8],
[L*P/8]])

Matrix([
[-L*P/8],
[-L*P/8]])

El valor de las redundantes en el sistema equivalente es:


Matrix([
[ 17*L*P/56],
[-29*L*P/56]])

In [67]:
# Calculamos las reacciones en el sistema original
AQ_loadFE=sp.Matrix([[P*L/4],
                     [-P*L/4+P*L/8]])

AQ_original=AQ+AQ_loadFE

print('Las redundantes en el sistema original son:')
display(AQ_original)

Las redundantes en el sistema original son:


Matrix([
[31*L*P/56],
[-9*L*P/14]])